In [1]:
import numpy as np
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, Concatenate
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """转换时间序列为监督学习格式"""
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):  # 输入序列 (t-n_in, ..., t-1)
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    for i in range(0, n_out):  # 预测序列 (t, t+1, ...)
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg


class MultiModalCurrencyLSTMModel:
    def __init__(self, price_path, sentiment_path, look_back=10):
        self.price_path = price_path
        self.sentiment_path = sentiment_path
        self.look_back = look_back
        # 分别为价格和情绪量表
        self.scaler_price = MinMaxScaler(feature_range=(0, 1))
        self.scaler_sentiment = MinMaxScaler(feature_range=(0, 1))
        self.scaler_y = MinMaxScaler(feature_range=(0, 1))  # 仅对y做归一化
        self.model = None

    def load_and_prepare_data(self):
        # 1. 读取CSV数据
        price_df = read_csv(self.price_path, header=0, index_col=0)
        sentiment_df = read_csv(self.sentiment_path, header=0, index_col=0)

        price_values = price_df.values.astype('float32')  # 价格序列
        sentiment_values = sentiment_df.values.astype('float32')  # 情绪序列

        # 2. 分别归一化
        price_scaled = self.scaler_price.fit_transform(price_values)
        sentiment_scaled = self.scaler_sentiment.fit_transform(sentiment_values)

        # 3. 转为监督学习格式
        price_supervised = series_to_supervised(price_scaled, self.look_back, 1)
        sentiment_supervised = series_to_supervised(sentiment_scaled, self.look_back, 1)

        # 4. 行数对齐（取最短长度），保证对应数据匹配
        min_len = min(len(price_supervised), len(sentiment_supervised))
        price_supervised = price_supervised.iloc[-min_len:]
        sentiment_supervised = sentiment_supervised.iloc[-min_len:]

        price_values = price_supervised.values
        sentiment_values = sentiment_supervised.values

        # 5. 分训练集和测试集（默认70%训练）
        train_size = int(min_len * 0.7)
        price_train = price_values[:train_size, :]
        price_test = price_values[train_size:, :]
        sentiment_train = sentiment_values[:train_size, :]
        sentiment_test = sentiment_values[train_size:, :]

        # 6. 拆分为X特征和y目标
        price_train_X, price_train_y = price_train[:, :-1], price_train[:, -1]
        price_test_X, price_test_y = price_test[:, :-1], price_test[:, -1]
        sentiment_train_X = sentiment_train[:, :-1]
        sentiment_test_X = sentiment_test[:, :-1]

        # 7. 目标y归一化，方便模型训练
        price_train_y = price_train_y.reshape(-1, 1)
        price_test_y = price_test_y.reshape(-1, 1)
        self.scaler_y.fit(price_train_y)
        price_train_y = self.scaler_y.transform(price_train_y)
        price_test_y = self.scaler_y.transform(price_test_y)

        # 8. 形状重塑成RNN(样本数, 时间步长, 特征数=1)
        price_train_X = price_train_X.reshape((price_train_X.shape[0], self.look_back, 1))
        price_test_X = price_test_X.reshape((price_test_X.shape[0], self.look_back, 1))
        sentiment_train_X = sentiment_train_X.reshape((sentiment_train_X.shape[0], self.look_back, 1))
        sentiment_test_X = sentiment_test_X.reshape((sentiment_test_X.shape[0], self.look_back, 1))

        return (price_train_X, sentiment_train_X, price_train_y), (price_test_X, sentiment_test_X, price_test_y)

    def build_model(self):
        # 价格输入分支
        price_input = Input(shape=(self.look_back, 1), name="price_input")
        price_lstm = LSTM(50, return_sequences=True, kernel_regularizer=l2(0.01))(price_input)
        price_lstm = Dropout(0.3)(price_lstm)
        price_lstm = LSTM(100, return_sequences=False, kernel_regularizer=l2(0.01))(price_lstm)
        price_lstm = Dropout(0.3)(price_lstm)

        # 情绪输入分支
        sentiment_input = Input(shape=(self.look_back, 1), name="sentiment_input")
        sentiment_lstm = LSTM(30, return_sequences=True, kernel_regularizer=l2(0.01))(sentiment_input)
        sentiment_lstm = Dropout(0.3)(sentiment_lstm)
        sentiment_lstm = LSTM(60, return_sequences=False, kernel_regularizer=l2(0.01))(sentiment_lstm)
        sentiment_lstm = Dropout(0.3)(sentiment_lstm)

        # 融合
        merged = Concatenate()([price_lstm, sentiment_lstm])

        # 全连接层
        dense1 = Dense(50, activation='relu')(merged)
        dense2 = Dropout(0.3)(dense1)
        output = Dense(1)(dense2)

        model = Model(inputs=[price_input, sentiment_input], outputs=output)
        model.compile(loss='mae', optimizer='adam')
        self.model = model
        return model

    def train(self, epochs=100, batch_size=64):
        (price_train_X, sentiment_train_X, train_y), (price_test_X, sentiment_test_X, test_y) = self.load_and_prepare_data()
        self.build_model()

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

        history = self.model.fit(
            [price_train_X, sentiment_train_X], train_y,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=([price_test_X, sentiment_test_X], test_y),
            verbose=2,
            shuffle=False,
            callbacks=[early_stopping, lr_scheduler]
        )
        return history

    def predict(self, price_X, sentiment_X):
        yhat = self.model.predict([price_X, sentiment_X])
        # 逆归一化输出
        yhat_inv = self.scaler_y.inverse_transform(yhat)
        return yhat_inv




In [ ]:
model = MultiModalCurrencyLSTMModel('CNY_JPY_to_exchange_rate.csv', 'JPY_sentiment.csv', look_back=10)
history = model.train(epochs=500, batch_size=64)

(price_train_X, sentiment_train_X, train_y), (price_test_X, sentiment_test_X, test_y) = model.load_and_prepare_data()
predictions = model.predict(price_test_X, sentiment_test_X)

from sklearn.metrics import mean_absolute_error, mean_squared_error
y_true = model.scaler_y.inverse_transform(test_y.reshape(-1, 1))
mae = mean_absolute_error(y_true, predictions)
rmse = np.sqrt(mean_squared_error(y_true, predictions))
print(f'MAE: {mae:.4f}, RMSE: {rmse:.4f}')

# 输出预测结果
print("模型对测试集的预测结果：")
print(predictions.flatten())

# 可选：对比真实值
y_true = model.scaler_y.inverse_transform(test_y.reshape(-1, 1))
print("测试集真实汇率：")
print(y_true.flatten())

Epoch 1/500
4/4 - 3s - 718ms/step - loss: 1.7384 - val_loss: 1.9017 - learning_rate: 0.0010
Epoch 2/500
4/4 - 0s - 22ms/step - loss: 1.5095 - val_loss: 1.5412 - learning_rate: 0.0010
Epoch 3/500
4/4 - 0s - 22ms/step - loss: 1.3825 - val_loss: 1.2932 - learning_rate: 0.0010
Epoch 4/500
4/4 - 0s - 23ms/step - loss: 1.2569 - val_loss: 1.2383 - learning_rate: 0.0010
Epoch 5/500
4/4 - 0s - 23ms/step - loss: 1.1431 - val_loss: 1.2061 - learning_rate: 0.0010
Epoch 6/500
4/4 - 0s - 23ms/step - loss: 1.0523 - val_loss: 1.0735 - learning_rate: 0.0010
Epoch 7/500
4/4 - 0s - 22ms/step - loss: 0.9536 - val_loss: 0.9044 - learning_rate: 0.0010
Epoch 8/500
4/4 - 0s - 21ms/step - loss: 0.8793 - val_loss: 0.8096 - learning_rate: 0.0010
Epoch 9/500
4/4 - 0s - 24ms/step - loss: 0.8048 - val_loss: 0.7595 - learning_rate: 0.0010
Epoch 10/500
4/4 - 0s - 21ms/step - loss: 0.7458 - val_loss: 0.7272 - learning_rate: 0.0010
Epoch 11/500
4/4 - 0s - 21ms/step - loss: 0.6861 - val_loss: 0.6382 - learning_rate: 0.0

: 

In [ ]:
import json
import pandas as pd
from datetime import datetime, timedelta

def create_prediction_api(model, currency_pair='JPY', days=20):
    """
    创建用于前端调用的预测API函数
    
    Args:
        model: 训练好的MultiModalCurrencyLSTMModel实例
        currency_pair: 货币对代码 (如 'JPY', 'HKD', 'SGD' 等)
        days: 预测天数
    
    Returns:
        dict: 包含预测结果的字典
    """
    try:
        # 获取最新的历史数据进行预测
        price_df = pd.read_csv(f'CNY_{currency_pair}_to_exchange_rate.csv', header=0, index_col=0)
        sentiment_df = pd.read_csv(f'{currency_pair}_sentiment.csv', header=0, index_col=0)
        
        # 获取最近的数据作为输入
        recent_price = price_df.values[-model.look_back:].astype('float32')
        recent_sentiment = sentiment_df.values[-model.look_back:].astype('float32')
        
        # 数据预处理 - 使用已训练好的scaler
        price_scaled = model.scaler_price.transform(recent_price)
        sentiment_scaled = model.scaler_sentiment.transform(recent_sentiment)
        
        # 重塑为模型输入格式
        price_input = price_scaled.reshape(1, model.look_back, 1)
        sentiment_input = sentiment_scaled.reshape(1, model.look_back, 1)
        
        # 生成多天预测
        predictions = []
        current_price_seq = price_input.copy()
        current_sentiment_seq = sentiment_input.copy()
        
        base_date = datetime.now()
        
        for i in range(days):
            # 预测下一天
            pred = model.predict(current_price_seq, current_sentiment_seq)
            pred_value = float(pred[0, 0])
            
            # 添加到结果列表
            pred_date = (base_date + timedelta(days=i+1)).strftime('%Y-%m-%d')
            predictions.append({
                'date': pred_date,
                'rate': round(pred_value, 4),
                'timestamp': int((base_date + timedelta(days=i+1)).timestamp() * 1000)
            })
            
            # 更新输入序列 - 将预测值作为下一次的输入
            # 对预测值进行归一化
            pred_normalized = model.scaler_price.transform([[pred_value]])
            
            # 滚动更新价格序列
            new_price_seq = np.roll(current_price_seq, -1, axis=1)
            new_price_seq[0, -1, 0] = pred_normalized[0, 0]
            current_price_seq = new_price_seq
            
            # 情感序列保持最后一个值（简化处理）
            new_sentiment_seq = np.roll(current_sentiment_seq, -1, axis=1)
            new_sentiment_seq[0, -1, 0] = current_sentiment_seq[0, -1, 0]
            current_sentiment_seq = new_sentiment_seq
        
        # 找出最佳购买时机（汇率最高点）
        rates = [p['rate'] for p in predictions]
        max_rate_idx = np.argmax(rates)
        predictions[max_rate_idx]['isOptimal'] = True
        
        return {
            'success': True,
            'currency_pair': f'CNY_{currency_pair}',
            'predictions': predictions,
            'model_info': {
                'model_type': 'LSTM+Sentiment',
                'look_back': model.look_back,
                'prediction_days': days
            }
        }
        
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'currency_pair': f'CNY_{currency_pair}'
        }

# 训练好模型后，创建预测函数
def predict_exchange_rate(currency_pair, days=20):
    """
    对外提供的预测接口
    """
    try:
        # 加载对应的模型（这里假设模型已经训练并保存）
        model = MultiModalCurrencyLSTMModel(
            f'CNY_{currency_pair}_to_exchange_rate.csv', 
            f'{currency_pair}_sentiment.csv', 
            look_back=10
        )
        
        # 如果模型文件存在，加载训练好的模型
        try:
            model.build_model()
            model.model.load_weights(f'{currency_pair}_lstm_model.h5')
        except:
            # 如果没有保存的模型，重新训练
            print(f"No saved model found for {currency_pair}, training new model...")
            model.train(epochs=100, batch_size=64)
            # 保存模型
            model.model.save_weights(f'{currency_pair}_lstm_model.h5')
        
        return create_prediction_api(model, currency_pair, days)
        
    except Exception as e:
        return {
            'success': False,
            'error': f'Failed to load model for {currency_pair}: {str(e)}'
        }

print("预测API函数已创建完成！")
print("可以通过 predict_exchange_rate('JPY', 20) 来调用预测功能")